In [1]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
import sys
from batteryml.pipeline import Pipeline
from batteryml.visualization.plot_helper import plot_capacity_degradation, plot_cycle_attribute, plot_result

%load_ext autoreload
%autoreload 2

# Extract SOH (State of Health) label from Cycle Capacity data

## Create Pipeline

In [ ]:
pipeline = Pipeline(config_path='configs/soh/dummy/matr_1.yaml',
                    workspace='workspaces')

workspaces


## Train and evaluate

In [3]:
model, dataset = pipeline.train(device='cuda', skip_if_executed=False)

Seed is set to 0.


Extracting features: 100%|██████████| 42/42 [00:00<00:00, 1289.36it/s]


In [ ]:
# get train test loss
train_prediction = model.predict(dataset, data_type='train')
train_loss = dataset.evaluate(train_prediction, 'RMSE', data_type='train') 
test_prediction = model.predict(dataset, data_type='test')
test_loss = dataset.evaluate(test_prediction, 'RMSE', data_type='test')
print(f'RMSE: Train {train_loss:.3f}, test {test_loss:.3f}')

RMSE: Train 0.032, test 0.026


In [5]:
# just evaluate using pipeline
pipeline.evaluate(model=model, dataset=dataset, skip_if_executed=False)

Seed is set to 0.
{'RMSE': 0.026197524741292}


# Extract SOH label from the SOH test file provided separately

## generate example soh test file
- We assume each battery cell undergoes a State of Health (SOH) test over 1000 cycles under standard conditions every 100 cycles. 
- The testing is performed at a controlled temperature, typically around 25°C, and utilizes a standardized discharge rate, such as C/2 or 1C.


And example soh file like this:
```
{'cell1': {100: {'relative': 0.92,
   'absolute': 1.01},
  200: {'relative': 0.89, 'absolute': 0.98},
  300: {'relative': 0.72, 'absolute': 0.80},
  400: {'relative': 0.71, 'absolute': 0.78},
  500: {'relative': 0.56, 'absolute': 0.62},
  600: {'relative': 0.69, 'absolute': 0.76},
  700: {'relative': 0.37, 'absolute': 0.40},
  800: {'relative': 0.38, 'absolute': 0.41},
  900: {'relative': 0.54, 'absolute': 0.59},
  1000: {'relative': 0.47, 'absolute': 0.52}
  },
  'cell2': {...}
  ...
}

```

In [4]:
# all cell_names in MATR1 dataset
cell_names = [
    'b1c1',  'b1c3',  'b1c5',  'b1c7',  'b1c11', 'b1c15',
    'b1c17', 'b1c19', 'b1c21', 'b1c24', 'b1c26', 'b1c28',
    'b1c30', 'b1c32', 'b1c34', 'b1c36', 'b1c38', 'b1c40',
    'b1c42', 'b1c44', 'b2c0',  'b2c2',  'b2c4',  'b2c6',
    'b2c11', 'b2c13', 'b2c17', 'b2c19', 'b2c21', 'b2c23',
    'b2c25', 'b2c27', 'b2c29', 'b2c31', 'b2c33', 'b2c35',
    'b2c37', 'b2c39', 'b2c41', 'b2c43', 'b2c45',

    'b1c0',  'b1c2',  'b1c4',  'b1c6',  'b1c9',  'b1c14',
    'b1c16', 'b1c18', 'b1c20', 'b1c23', 'b1c25', 'b1c27',
    'b1c29', 'b1c31', 'b1c33', 'b1c35', 'b1c37', 'b1c39',
    'b1c41', 'b1c43', 'b1c45', 'b2c1',  'b2c3',  'b2c5',
    'b2c10', 'b2c12', 'b2c14', 'b2c18', 'b2c20', 'b2c22',
    'b2c24', 'b2c26', 'b2c28', 'b2c30', 'b2c32', 'b2c34',
    'b2c36', 'b2c38', 'b2c40', 'b2c42', 'b2c44', 'b2c46',
    'b2c47'
]

# We assume each battery cell undergoes a State of Health (SOH) test over 1000 cycles under standard conditions every 100 cycles. 
# The testing is performed at a controlled temperature, typically around 25°C, and utilizes a standardized discharge rate, such as C/2 or 1C.
import random
soh_dict = {}
cycles = [cycle for cycle in range(100, 1001, 100)]
print(cycles)

# Function to simulate SOH degradation
def simulate_soh_degradation(start_soh, cycle):
    degradation_rate = random.uniform(0.0005, 0.001)  # Random degradation rate per cycle
    return max(start_soh - degradation_rate * cycle, 0)  # Ensure SOH doesn't go below 0

# Generate SOH for each cell and each cycle
for name in cell_names:
    cell_name = f'MATR_{name}'
    soh_dict[cell_name] = {}
    start_soh = 1.0  # Assume starting SOH is 1.0
    for cycle in cycles:
        # soh_dict[cell_name][cycle] = {"relative": simulate_soh_degradation(start_soh, cycle)}
        nominal_capacity = 1.1
        relative_soh = simulate_soh_degradation(start_soh, cycle)
        soh_dict[cell_name][cycle] = {
            "relative": relative_soh,
            "absolute": relative_soh * nominal_capacity
        }

import json
# Save the SOH dictionary to a JSON file
with open('data/example_soh_dict.json', 'w') as json_file:
    json.dump(soh_dict, json_file, indent=4)


[100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]


## Create pipeline

In [6]:
pipeline = Pipeline(config_path='configs/soh/dummy/matr_1_with_soh_filepath.yaml',
                    workspace='workspaces')

workspaces


In [7]:
model, dataset = pipeline.train(device='cuda', skip_if_executed=False)

Seed is set to 0.
read soh file


Extracting features: 100%|██████████| 42/42 [00:00<00:00, 1296.07it/s]


In [ ]:
# get train test loss
train_prediction = model.predict(dataset, data_type='train')
train_loss = dataset.evaluate(train_prediction, 'RMSE', data_type='train') 
test_prediction = model.predict(dataset, data_type='test')
test_loss = dataset.evaluate(test_prediction, 'RMSE', data_type='test')
print(f'RMSE: Train {train_loss:.3f}, test {test_loss:.3f}')

RMSE: Train 0.046, test 0.043


In [9]:
# just evaluate using pipeline
pipeline.evaluate(model=model, dataset=dataset, skip_if_executed=False)

Seed is set to 0.
{'RMSE': 0.04273472726345062}
